<a href="https://colab.research.google.com/github/sho-portfolio/MachineLearning-MultiClassClassifier/blob/master/GoogleColab/main-LocalRuntime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
"""
https://research.google.com/colaboratory/local-runtimes.html

author:       sho 
date:         2019-11-01
medium:       ()
github:       (https://github.com/sho-portfolio/MachineLearning-MultiClassClassifier)
donate:       ()
description:  code to train, evaluate, productionize and use a model to predict in ~35 lines of code
              four main sections:
              [1] prepare the data
              [2] train and evaluate the model
              [3] productionize the model (save the model and associated files)
              [4] use the saved model to make predictions
"""

In [6]:
!pip3 install jupyter_http_over_ws


In [8]:
!jupyter serverextension enable --py jupyter_http_over_ws

Traceback (most recent call last):
  File "/usr/local/bin/jupyter-serverextension", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python2.7/dist-packages/jupyter_core/application.py", line 267, in launch_instance
    return super(JupyterApp, cls).launch_instance(argv=argv, **kwargs)
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/notebook/serverextensions.py", line 293, in start
    super(ServerExtensionApp, self).start()
  File "/usr/local/lib/python2.7/dist-packages/jupyter_core/application.py", line 256, in start
    self.subapp.start()
  File "/usr/local/lib/python2.7/dist-packages/notebook/serverextensions.py", line 210, in start
    self.toggle_server_extension_python(arg)
  File "/usr/local/lib/python2.7/dist-packages/notebook/serverextensions.py", line 199, in toggle_server_extension_python
    m, server_exts = _get_server_extension_meta

In [9]:
!jupyter notebook \
  --NotebookApp.allow_origin='https://colab.research.google.com' \
  --port=8888 \
  --NotebookApp.port_retries=0

/usr/local/lib/python2.7/dist-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")
Traceback (most recent call last):
  File "/usr/local/bin/jupyter-notebook", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python2.7/dist-packages/jupyter_core/application.py", line 267, in launch_instance
    return super(JupyterApp, cls).launch_instance(argv=argv, **kwargs)
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 657, in launch_instance
    app.initialize(argv)
  File "</usr/local/lib/python2.7/dist-packages/decorator.pyc:decorator-gen-7>", line 2, in initialize
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 87, in catch_config_error
    return method(app, *args, **kwargs)
  File "/usr/local/lib/python2.7/dist-packages/notebook/notebookapp.py", line 1368, in 

In [0]:
!jupyter notebook \
  --NotebookApp.allow_origin='https://colab.research.google.com' \
  --ip=127.0.0.1 \
  --NotebookApp.port_retries=0

/usr/local/lib/python2.7/dist-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")
|INFO|google.colab serverextension initialized.
|INFO|Serving notebooks from local directory: /content
|INFO|0 active kernels
|INFO|The Jupyter Notebook is running at:
|INFO|http://127.0.0.1:8888/
|INFO|Use Control-C to stop this server and shut down all kernels (twice to skip confirmation).


In [0]:
#download data source then use it (get single file)
!wget "https://raw.githubusercontent.com/sho-portfolio/MachineLearning-MultiClassClassifier/master/helper.py"

#Simple Data Example
urlTrain = "https://raw.githubusercontent.com/sho-portfolio/MachineLearning-MultiClassClassifier/master/dataTrain.txt"
urlTest = "https://raw.githubusercontent.com/sho-portfolio/MachineLearning-MultiClassClassifier/master/dataTest.txt"

#Stack Overflow Data Example
urlTrain = "https://raw.githubusercontent.com/sho-portfolio/MachineLearning-MultiClassClassifier/master/data/StackOverflow_dataTrain.csv"
urlTest = "https://raw.githubusercontent.com/sho-portfolio/MachineLearning-MultiClassClassifier/master/data/StackOverflow_dataTest.csv"


--2019-11-04 22:34:27--  https://raw.githubusercontent.com/sho-portfolio/MachineLearning-MultiClassClassifier/master/helper.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3357 (3.3K) [text/plain]
Saving to: ‘helper.py.1’

helper.py.1         100%[===================>]   3.28K  --.-KB/s    in 0s      

2019-11-04 22:34:27 (28.5 MB/s) - ‘helper.py.1’ saved [3357/3357]



In [0]:
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing import sequence
from keras.models import load_model
from keras import metrics
from keras import utils
from subprocess import check_output
import pickle
import helper as helper #only used if dataset is unbalanced or not optimized for ML

Using TensorFlow backend.


In [0]:
## get and prepare data

# load the dataset
df = pd.read_csv(urlTrain)
#X_train, y_train, X_test, y_test = helper.load_dataset('dataTrain.txt', 'text', 'label', 0.8)
print(df)

#texts = df['text']
#labels = df['label']
texts = df['post'] #use this for the #stack-overflow dataset
labels = df['tags'] #use this for the #stack-overflow dataset

                                                    post     tags
0      sql: why is there a need for subqueries in thi...      sql
1      counting days left live  i am creating a small...  android
2      android deadsystemexception  currently we are ...  android
3      passing data from sqlitedatabase to a detailvi...      ios
4      what could be the expression  double (f)(doubl...        c
...                                                  ...      ...
34994  how to display sub folder name using php  i wa...      php
34995  mfmailcomposeviewcontroller displaying only ba...   iphone
34996  how to make uiimageview as a circle [ios]  i h...      ios
34997  does it matter which order i use for my css bo...      css
34998  how to refresh a page after i perform a loopin...  asp.net

[34999 rows x 2 columns]


In [0]:
# Split data into train and test (80% used for training, 20% for validation) (X = text Y = label)
train_size = int(len(df) * 0.6)
X_train = texts[:train_size]
y_train = labels[:train_size]
X_test = texts[train_size:]
y_test = labels[train_size:]

In [0]:
# prepare (encode) model input data (using tokenizer)
tok = Tokenizer()
tok.fit_on_texts(X_train)
X_train_enc = tok.texts_to_matrix(X_train, mode='count')
X_test_enc = tok.texts_to_matrix(X_test, mode='count')

In [0]:
# prepare (encode) target data (the labels that should be predicted)
num_classes = y_train.nunique()
le = LabelEncoder()
le.fit_transform(y_train)
y_train_enc = le.transform(y_train)
y_test_enc = le.transform(y_test) #test labels transformed but not fit
y_train_enc_categorical = utils.to_categorical(y_train_enc, num_classes)
y_test_enc_categorical = utils.to_categorical(y_test_enc, num_classes)

NameError: ignored

In [0]:
#print("\n X_train_enc: \n", X_train_enc)
#print("\n X_test_enc: \n", X_test_enc)
#print("\n y_train_enc: \n", y_train_enc)
#print("\n y_test_enc: \n", y_test_enc)
#print("\n y_train_enc_categorical: \n", y_train_enc_categorical)
#print("\n y_test_enc_categorical: \n", y_test_enc_categorical)

In [0]:
#specify the model parameters
epochs=40
number_of_neurons = 500
batch_size = 128
dropout_fraction = 0.5
vocab_size = min(len(tok.word_index) + 1, 1000)

In [0]:
#define the model
model = Sequential()
model.add(Dense(number_of_neurons, input_shape=(vocab_size,), activation='relu'))
model.add(Dropout(dropout_fraction, seed=None))
model.add(Dense(num_classes)) 
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[metrics.categorical_accuracy])
print(model.summary())




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 500)               5500      
_________________________________________________________________
dropout_1 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 1503      
_________________________________________________________________
activation_1 (Activation)    (None, 3)                 0         
Total params: 7,003
Trainable params: 7,003
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
# fit the model
model.fit(X_train_enc, y_train_enc_categorical, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0.2)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 3 samples, validate on 1 samples
Epoch 1/40





3/3 [==============================] - 1s 313ms/step - loss: 1.0497 - categorical_accuracy: 0.3333 - val_loss: 0.8872 - val_categorical_accuracy: 1.0000
Epoch 2/40
3/3 [==============================] - 0s 2ms/step - loss: 1.0661 - categorical_accuracy: 0.3333 - val_loss: 0.8626 - val_categorical_accuracy: 1.0000
Epoch 3/40
3/3 [==============================] - 0s 3ms/step - loss: 0.9253 - categorical_accuracy: 1.0000 - val_loss: 0.8362 - val_categorical_accuracy: 1.0000
Epoch 4/40
3/3 [==============================] - 0s 1ms/step - loss: 1.0407 - categorical_accuracy: 0.6667 - val_loss: 0.8090 - val_categorical_accuracy: 1.0000
Epoch 5/40
3/3 [==============================] - 0s 1ms/step - loss: 0.8774 - categorical_accuracy: 1.0000 - val_loss: 0.7813 - val_categorical_accuracy: 1.0000
Epoch 6/40
3/3 [============================

In [0]:

# evaluate the accuracy of the trained model
score = model.evaluate(X_test_enc, y_test_enc_categorical, batch_size=batch_size, verbose=1)
print("score: ", score)

2/2 [==============================] - 0s 973us/step
score:  [0.20228077471256256, 1.0]


In [0]:
## Productionize the model

#save the model (so that it can be used later)
model.save('model.h5')

#save/Pickle the tokenizer
with open('tokenizer.pickle', 'wb') as handle: pickle.dump(tok, handle, protocol=pickle.HIGHEST_PROTOCOL)

#save/Pickle the Label Encoder
with open('labelencoder.pickle', 'wb') as handle: pickle.dump(le, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:

## Use the saved model to make predictions

In [0]:
#load the model
model = load_model('model.h5')

#load the label encoder
with open('labelencoder.pickle', 'rb') as handle: le = pickle.load(handle)

# load the tokenizer
with open('tokenizer.pickle', 'rb') as handle: tok = pickle.load(handle)

In [0]:
#create a dataframe from the test datafile
df = pd.read_csv(urlTest)
#df = pd.read_csv('data/dataTestSO.txt') #stack-overflow dataset

#encode the text values we want to predict on using the loaded tokenizer pickle file
X_test = df['text']
X_test_enc = tok.texts_to_matrix(X_test, mode='count')

In [0]:
#predict values for the test data
#create the output dataframe
df_output = pd.DataFrame(columns=['text', 'label'])
text_labels = le.classes_

for i in range(0,X_test_enc.shape[0]):
  prediction = model.predict(np.array([X_test_enc[i]]))
  predicted_label = text_labels[np.argmax(prediction)]
  #print("input: ", X_test[i])
  #print("prediction: ", prediction)
  #print("predicted_label: ", predicted_label)
  df_output = df_output.append({'text': X_test.iloc[i], 'label':predicted_label}, ignore_index=True)

print (df_output)

         text label
0   containsA     A
1   containsC     C
2   containsB     B
3   containsB     B
4   containsA     A
5           A     A
6           B     B
7     letterA     A
8         cat     B
9         dog     B
10       insA     B
11       insB     B


In [0]:
#See error logs
!cat /var/log/colab-jupyter.log

import json

with open("/var/log/colab-jupyter.log", "r") as fo:
  for line in fo:
    print(json.loads(line)['msg'])

{"name":"app","hostname":"1c5690183a6c","pid":10,"type":"jupyter","level":40,"msg":"/usr/local/lib/python2.7/dist-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.","time":"2019-11-04T21:50:53.829Z","v":0}
{"name":"app","hostname":"1c5690183a6c","pid":10,"type":"jupyter","level":40,"msg":"  warn(\"IPython.utils.traitlets has moved to a top-level traitlets package.\")","time":"2019-11-04T21:50:53.830Z","v":0}
{"name":"app","hostname":"1c5690183a6c","pid":10,"type":"jupyter","level":30,"msg":"Writing notebook server cookie secret to /root/.local/share/jupyter/runtime/notebook_cookie_secret","time":"2019-11-04T21:50:55.138Z","v":0}
{"name":"app","hostname":"1c5690183a6c","pid":10,"type":"jupyter","level":30,"msg":"google.colab serverextension initialized.","time":"2019-11-04T21:50:55.246Z","v":0}
{"name":"app","hostname":"1c5690183a6c","pid":10,"type":"jupyter","level":30,"msg":"Serving notebooks from local directory: /

In [0]:
!pip install --upgrade jupyter_http_over_ws>=0.0.1a3 && \
  jupyter serverextension enable --py jupyter_http_over_ws

In [0]:

#https://drive.google.com/file/d/1ijawhI6AuPXxWhLu8flkrHZPTu3o_zF0/view?usp=sharing

!nvidia-smi

Mon Nov  4 22:17:04 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.50       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
!gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?code_challenge=zLY6MbvCJWekizLYCbdaZey1UJTFqjVOtk9V4v7Xj2M&prompt=select_account&code_challenge_method=S256&access_type=offline&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&client_id=32555940559.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth


Enter verification code: 4/swHVaBVsNmKz6HWCQ-TX0PJTkha0baD5Jiw6ztO-VRUXDacygnn0Wds
If you need to use ADC, see:
  gcloud auth application-default --help

You are now logged in as [sho.munshi@gmail.com].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [0]:
!gcloud projects list


PROJECT_ID             NAME              PROJECT_NUMBER
arctic-compass-223217  My First Project  611112547936
pythonsho              PythonSho         282401377659


In [0]:
!gcloud config set project pythonsho 

Updated property [core/project].


In [0]:
!gcloud compute instances list

API [compute.googleapis.com] not enabled on project [282401377659]. 
Would you like to enable and retry (this will take a few minutes)? 
(y/N)?  y

Enabling service [compute.googleapis.com] on project [282401377659]...
Operation "operations/acf.e1d9930c-e4f3-4967-bffe-a898de4c1ad2" finished successfully.
Listed 0 items.


In [0]:
!gcloud compute instances list

Listed 0 items.


In [0]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn't guaranteed
gpu = GPUs[0]

def printm():
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " I Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print('GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB'.format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))

printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7410 sha256=27834128d12dd023aab7a335c8f661045fdb25493622598acf47c2d168637c65
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 26.4 GB  I Proc size: 159.4 MB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB
